# Basic run

First, we import the essential libraries and modules that will be used throughout this tutorial:

- `pickle`: Used for saving and loading Python objects to and from files.
- `Energyscope` from `energyscope.energyscope`: The main class for initializing and running the EnergyScope model.
- `infrastructure_ch_2050`, `Model` from `energyscope.models`: The specific model configuration we will use, which focuses on energy infrastructure. And the object Model to create your own model from external files.
- `postprocessing` from `energyscope.result`: Functions for processing and analyzing the results after optimization.
- `plot_sankey` from `energyscope.plots`: A function to generate Sankey diagrams for visualizing energy flows.

In [1]:
# This cell serves only to display a button that allows the user to download the AMPL files used in this tutorial. You can discard it if you run the .ipynb file locally.

from IPython.display import HTML

button_html = """
<div style="text-align: center;">
<button onclick="openMultipleLinks()" 
        style='padding: 10px 20px; background-color: #4CAF50; color: white; 
               border: none; border-radius: 4px; cursor: pointer;'>
  Download AMPL files
</button>

<script>
  function openMultipleLinks() {
    // List of URLs you want to open
    var urls = [
      'https://gitlab.com/energyscope/energyscope/-/raw/19-documentation-add-tuto-direct-download-link-for-ipynb-and-ampl-files/src/energyscope/data/models/infrastructure/switzerland/ses_main.mod?inline=false',
      'https://gitlab.com/energyscope/energyscope/-/raw/19-documentation-add-tuto-direct-download-link-for-ipynb-and-ampl-files/src/energyscope/data/datasets/infrastructure/switzerland/2050/data.dat?inline=false',
      'https://gitlab.com/energyscope/energyscope/-/raw/19-documentation-add-tuto-direct-download-link-for-ipynb-and-ampl-files/src/energyscope/data/datasets/infrastructure/switzerland/2050/techs.dat?inline=false'
    ];
    
    // Open each URL with a 300ms delay between them
    for (var i = 0; i < urls.length; i++) {
      setTimeout(function(url) {
        window.open(url, '_blank');
      }, i * 300, urls[i]);
    }
  }
</script>
"""

HTML(button_html)


In [2]:
import pickle
from energyscope.energyscope import Energyscope
from energyscope.models import infrastructure_ch_2050, Model
from energyscope.result import postprocessing
from energyscope.plots import plot_sankey

---
## Initialize and Run the Model

In this section, we initialize the EnergyScope model using the infrastructure dataset and perform a single optimization run.

### Initialize the Model

Create an instance of the `Energyscope` class with the `infrastructure` model. This sets up the model with predefined parameters and datasets.

In [3]:
es_infra_ch = Energyscope(model=infrastructure_ch_2050)

### Export Model to AMPL and GLPK

For compatibility and further analysis, we export the model files in both AMPL and GLPK formats.

In [4]:
# Export model to AMPL
es_infra_ch.export_ampl(mod_filename='tutorial_output/AMPL_infrastructure_ch_2050.mod',dat_filename='tutorial_output/AMPL_infrastructure_ch_2050.dat')

# Export model to GLPK
es_infra_ch.export_glpk(mod_filename='tutorial_output/GLPK_infrastructure_ch_2050.mod',dat_filename='tutorial_output/GLPK_infrastructure_ch_2050.dat')

### Load External AMPL files

To load external files from AMPL you need to create a new Model as follow.

In [5]:
# Create you own Model object from imported AMPL files
Model_es_infra_ch = Model([
    ('mod', "tutorial_output/AMPL_infrastructure_ch_2050.mod"),
    ('dat', "tutorial_output/AMPL_infrastructure_ch_2050.dat")
])

# Create an instance of the Energyscope class with your own model.
es_infra_ch = Energyscope(model=Model_es_infra_ch)

### Solve the Model
Perform the optimization calculation. This step runs the solver and computes the optimal configuration based on the model.

In [6]:
results_ch = es_infra_ch.calc()

Gurobi 11.0.0:Gurobi 11.0.0: optimal solution; objective 9229.741872
5670 simplex iteration(s)
1 branching node(s)



### Post-Process Results
After obtaining the raw results, we apply post-processing to compute Key Performance Indicators (KPIs) and prepare the data for visualization.

In [7]:
results_ch = postprocessing(results_ch)

# Example of how to extract post-processed data
results_ch.postprocessing['df_annual'].loc['WIND',:]

,C_inv,C_maint,Annual_Prod,F_Mult,tau,C_op,C_inv_an,Annual_Use,Category,Category_2,Sector
Run,,,,,,,,,,,
0,29312.4,458.0,40299.36,20.0,0.062433,0.0,1830.059062,40299.36,ELECTRICITY_MV,Wind,Electricity


---

## Visualize Results with a Sankey Diagram

A Sankey diagram is an effective way to visualize energy flows within the system.

### Generate the Sankey Diagram
Use the `plot_sankey` function with the processed results to create the diagram.



In [8]:
fig = plot_sankey(results_ch)

### Display the Diagram
Render the Sankey diagram within the notebook for immediate visualization.
   > **Optional**: You can save the diagram as an HTML file or an image for external use by uncommenting the following lines:

In [9]:
# fig.write_html("tutorial_output/sankey_infrastructure_ch_2050.html")
# fig.write_image('tutorial_output/sankey_infrastructure_ch_2050.png')

In [10]:
fig.show()

---

## Save and Load Results

To preserve the results and avoid re-running the optimization every time, we can save the results to a file and load them later.

### Define Save Function
Create a function to save the `Result` object using `pickle`.



In [11]:
def save_result_to_pickle(data, filename):
    """
    Save the Result object to a pickle file.

    Parameters:
        data: The Result object to save.
        filename (str): The file path to save the object to.
    """
    with open(filename, 'wb') as fp:
        pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)

### Define Load Function
Create a function to load the `Result` object from a pickle file.


   > **Note**: These utility functions could be integrated into the EnergyScope library for convenience.



In [12]:
def load_result_from_pickle(filename):
    """
    Load the Result object from a pickle file.

    Parameters:
        filename (str): The file path to load the object from.

    Returns:
        The loaded Result object.
    """
    with open(filename, 'rb') as handle:
        result = pickle.load(handle)
    return result

### Save the Results
Use the `save_result_to_pickle` function to save the results to a file.



In [13]:
save_result_to_pickle(results_ch, "tutorial_input/infrastructure_ch_2050.pickle")

### Clear the Results Variable
Empty the `results_ch` variable to simulate a fresh environment.


In [14]:
results_ch = None

### Load the Results
Load the previously saved results using the `load_result_from_pickle` function.



In [15]:
results_ch = load_result_from_pickle("tutorial_input/infrastructure_ch_2050.pickle")

### Display Total Cost
Access and display the total cost from the loaded results to verify that the data was correctly saved and loaded.

 

In [16]:
results_ch.variables['TotalCost']

,TotalCost,Run
0,9229.741872,0


  This should output a table showing the total cost of the optimized energy system configuration.

---

By following these steps, you can perform a basic run of the EnergyScope model, visualize the results, and save/load the data for future use. This tutorial serves as a foundation for more complex analyses and customizations.